In [66]:
import Pkg
Pkg.activate("../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [67]:
using Revise
#using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
#using StatsPlots
using LinearAlgebra
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [68]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [69]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [70]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi, dy)

291×5 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬────────────────────┬─────────────┬─────────────────┬─────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI  │ Yeld_10Y_T_Bill │ CPI        ⋯
├────────────┼────────────────────┼─────────────┼─────────────────┼─────────────
│ 1952-03-01 │        -0.00772801 │  0.00772445 │       0.0022875 │ -0.0037807 ⋯
│ 1952-06-01 │          0.0651996 │ -0.00708528 │       0.0017875 │   0.011299 ⋯
│ 1952-09-01 │         -0.0227685 │  0.00423886 │       0.0024375 │         0. ⋯
│ 1952-12-01 │            0.08432 │   0.0089235 │       0.0011425 │ -0.0037523 ⋯
│ 1953-03-01 │         -0.0487217 │  0.00497517 │       0.0018125 │         0. ⋯
│ 1953-06-01 │        -0.00778441 │ -0.00227055 │       0.0021925 │  0.0074906 ⋯
│ 1953-09-01 │        -0.00277599 │ -0.00299956 │        0.002115 │  0.0074349 ⋯
│ 1953-12-01 │          0.0705234 │  0.00767892 │       0.0021725 │ -0.0037105 ⋯
│     ⋮      │         ⋮         

In [36]:
CSV.write("usa_data_transformed.csv", data)

UndefVarError: UndefVarError: `CSV` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [71]:
start = Date(1975,01,01)
end_d = Date(2020, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1395,0.8255,0.1138,0.2722,-1.0147,-0.6321,-0.0168,0.0909,1.6908,0.8155,0.1398


In [72]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.0701,0.1868,0.0175,0.0623,-0.2565,-0.5872,-0.0166,0.0868,1.3255,0.2501,0.0218
T_Bill_CPI,0.0083,0.0255,0.0019,0.0073,-0.0208,0.0704,-0.0038,0.3505,0.2942,0.2237,0.0024
Yeld_10Y_T_Bill,0.0031,0.0099,0.0043,0.0066,-0.0077,-0.7037,0.0021,0.769,0.7198,1.3822,0.0048
CPI,0.009,0.0403,0.0088,0.0124,-0.0254,0.3024,0.004,0.4101,2.0781,0.9769,0.0082
Dividend_Yeld,0.4315,-2.7747,-3.6729,-3.3352,-4.4867,0.2391,-4.0007,0.9781,-1.0182,-8.5116,-3.8214


In [73]:
returns_summarystats(s_data, freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.1402,0.7471,0.0702,0.2492,-1.026,-0.5872,-0.0662,0.0868,1.3255,0.5003,0.0873
T_Bill_CPI,0.0166,0.1019,0.0074,0.0291,-0.0832,0.0704,-0.0152,0.3505,0.2942,0.4474,0.0095
Yeld_10Y_T_Bill,0.0062,0.0397,0.0172,0.0264,-0.0306,-0.7037,0.0084,0.769,0.7198,2.7644,0.019
CPI,0.018,0.1612,0.0351,0.0496,-0.1016,0.3024,0.0159,0.4101,2.0781,1.9539,0.0326
Dividend_Yeld,0.863,-11.0987,-14.6915,-13.3408,-17.9466,0.2391,-16.0028,0.9781,-1.0182,-17.0231,-15.2855


In [74]:
s_data_cor = cor(values(s_data))
pretty_table(s_data_cor, backend = Val(:html),header=colnames(s_data), row_labels=colnames(s_data), title="correlatio matrix")

,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld
Total_Stock_T_Bill,1.0,-0.0600267,0.0491574,-0.0402203,-0.0897381
T_Bill_CPI,-0.0600267,1.0,-0.0795142,-0.530145,0.189184
Yeld_10Y_T_Bill,0.0491574,-0.0795142,1.0,-0.258195,0.0791524
CPI,-0.0402203,-0.530145,-0.258195,1.0,0.464042
Dividend_Yeld,-0.0897381,0.189184,0.0791524,0.464042,1.0


In [75]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld,std
Total_Stock_T_Bill,0.2481,0.0922,-2.1897,-3.4561,-3.1543,0.0507,0.0684
T_Bill_CPI,-0.0344,-0.0119,0.855,0.7645,0.7257,-0.0069,0.007
Yeld_10Y_T_Bill,0.0017,-0.0036,-0.002,0.7703,-0.0041,0.0002,0.002
CPI,0.0361,0.02,0.1018,-0.6647,0.2425,0.0074,0.0074
Dividend_Yeld,-0.1911,-0.0795,0.927,2.5259,2.2616,0.958,0.0713


,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,Dividend_Yeld
Total_Stock_T_Bill,1.0,0.03,0.08,-0.08,-0.97
T_Bill_CPI,0.03,1.0,-0.14,-0.95,-0.05
Yeld_10Y_T_Bill,0.08,-0.14,1.0,-0.11,-0.08
CPI,-0.08,-0.95,-0.11,1.0,0.09
Dividend_Yeld,-0.97,-0.05,-0.08,0.09,1.0


In [76]:
isposdef(Symmetric(vmodel.Σ))

true

In [77]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

MethodError: MethodError: no method matching Distributions.InverseWishart(::Int64, ::Symmetric{Float64, Matrix{Float64}})
The type `Distributions.InverseWishart` exists, but no method is defined for this combination of argument types when trying to construct it.

Closest candidates are:
  Distributions.InverseWishart(::Real, !Matched::Cholesky)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\YQSrn\src\matrix\inversewishart.jl:47
  Distributions.InverseWishart(::Real, !Matched::Matrix)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\YQSrn\src\matrix\inversewishart.jl:45
  Distributions.InverseWishart(!Matched::Tracker.TrackedReal, ::AbstractMatrix{<:Real})
   @ DistributionsADTrackerExt C:\Users\matsz\.julia\packages\DistributionsAD\tWlyT\ext\DistributionsADTrackerExt.jl:586
  ...


In [78]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.1837,-0.2099,-0.2143,-0.2251,-0.2216,-0.2231,-0.2304,-0.2281,-0.2223,-0.2321,-0.2252,-0.2277,-0.222,-0.2256,-0.2374
0.05,-0.1294,-0.1581,-0.1592,-0.1652,-0.1618,-0.1645,-0.17,-0.1704,-0.1652,-0.1754,-0.1701,-0.1694,-0.1682,-0.1671,-0.1695
0.25,0.0082,-0.0178,-0.0198,-0.0273,-0.0206,-0.0209,-0.027,-0.0289,-0.0233,-0.031,-0.0254,-0.027,-0.0272,-0.0303,-0.0264
0.5,0.0987,0.0807,0.0788,0.0753,0.0749,0.0773,0.072,0.0742,0.0759,0.0714,0.0748,0.0727,0.0739,0.0716,0.0722
0.75,0.1905,0.1781,0.1785,0.1742,0.1719,0.1749,0.1712,0.1746,0.1763,0.1697,0.1731,0.1717,0.1727,0.1713,0.1735
0.95,0.3268,0.3155,0.3202,0.3145,0.3211,0.3192,0.3138,0.3131,0.3199,0.3185,0.3184,0.3108,0.3155,0.3137,0.3151
0.98,0.39,0.3757,0.3833,0.3774,0.3825,0.3816,0.3709,0.3684,0.3809,0.3777,0.3747,0.3696,0.3761,0.3735,0.3787


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0494,-0.0512,-0.0524,-0.0527,-0.0542,-0.0561,-0.0549,-0.0554,-0.056,-0.0569,-0.0563,-0.0575,-0.0576,-0.0573,-0.0586
0.05,-0.0431,-0.043,-0.0434,-0.0449,-0.0455,-0.0469,-0.0462,-0.0466,-0.0468,-0.0471,-0.0473,-0.0484,-0.0485,-0.0488,-0.0498
0.25,-0.0284,-0.0255,-0.0248,-0.0252,-0.0251,-0.0261,-0.0259,-0.0262,-0.0262,-0.0263,-0.0264,-0.0262,-0.0264,-0.0267,-0.0265
0.5,-0.0183,-0.0137,-0.0121,-0.0114,-0.0109,-0.0117,-0.0113,-0.0115,-0.0118,-0.0117,-0.0112,-0.0115,-0.0114,-0.0111,-0.0113
0.75,-0.0079,-0.0015,0.001,0.0022,0.0028,0.0027,0.0032,0.0032,0.0029,0.0029,0.0036,0.0038,0.004,0.0043,0.004
0.95,0.0068,0.0157,0.0194,0.0216,0.023,0.0237,0.0241,0.0242,0.0239,0.0248,0.0251,0.0257,0.0266,0.0251,0.0266
0.98,0.0124,0.0229,0.0279,0.0301,0.0315,0.0331,0.0326,0.0327,0.0336,0.033,0.0334,0.0347,0.0357,0.0355,0.0358


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0071,-0.0082,-0.0071,-0.0077,-0.0071,-0.0072,-0.0072,-0.0074,-0.0073,-0.007,-0.0074,-0.0073,-0.0068,-0.0067,-0.0065
0.05,-0.0033,-0.004,-0.0031,-0.0031,-0.0027,-0.0025,-0.0025,-0.0027,-0.0025,-0.0021,-0.0025,-0.0027,-0.0024,-0.0022,-0.0022
0.25,0.0049,0.0063,0.0076,0.008,0.0081,0.0081,0.008,0.0083,0.0081,0.0081,0.0081,0.0079,0.0082,0.0082,0.0084
0.5,0.0107,0.0138,0.0149,0.0155,0.0156,0.0156,0.0155,0.0155,0.0156,0.0156,0.0153,0.0153,0.0154,0.0156,0.0155
0.75,0.0163,0.0211,0.0223,0.0228,0.0231,0.023,0.0228,0.0226,0.0229,0.023,0.0228,0.0225,0.0229,0.0229,0.0231
0.95,0.0244,0.0308,0.0328,0.0334,0.0338,0.0336,0.0337,0.0336,0.0335,0.0335,0.0332,0.0333,0.0337,0.0334,0.0337
0.98,0.0277,0.0353,0.0375,0.0377,0.0382,0.0381,0.0385,0.0383,0.0379,0.0379,0.0376,0.0379,0.0379,0.0381,0.0385


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0152,-0.0254,-0.0291,-0.0308,-0.0321,-0.0323,-0.0331,-0.0344,-0.0348,-0.0348,-0.0367,-0.0366,-0.0367,-0.0383,-0.0384
0.05,-0.008,-0.0169,-0.0205,-0.022,-0.0229,-0.0238,-0.0241,-0.0254,-0.0252,-0.0258,-0.0266,-0.0264,-0.0275,-0.0283,-0.0283
0.25,0.0092,0.0023,-0.0001,-0.0014,-0.0025,-0.0023,-0.0029,-0.0032,-0.0033,-0.0034,-0.0034,-0.0037,-0.0045,-0.0049,-0.0053
0.5,0.0211,0.0156,0.0137,0.0134,0.0122,0.0125,0.0125,0.0126,0.012,0.0122,0.0125,0.0122,0.0118,0.0117,0.0111
0.75,0.0328,0.0293,0.0279,0.0273,0.0274,0.0276,0.0271,0.0278,0.028,0.0277,0.028,0.0282,0.0283,0.0283,0.0279
0.95,0.0495,0.0485,0.0471,0.0473,0.0486,0.0494,0.049,0.0499,0.0506,0.0501,0.0496,0.0512,0.0514,0.0516,0.052
0.98,0.0583,0.0563,0.0564,0.0561,0.0577,0.0589,0.0585,0.0592,0.06,0.0594,0.0596,0.0606,0.0604,0.0615,0.0618


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-17.5858,-18.1781,-18.5596,-18.8247,-19.0009,-19.203,-19.2996,-19.431,-19.532,-19.6377,-19.7696,-19.8627,-19.966,-20.026,-20.0517
0.05,-17.4282,-17.8988,-18.1961,-18.4096,-18.5823,-18.7195,-18.8488,-18.952,-19.0414,-19.0975,-19.1467,-19.2536,-19.3269,-19.3645,-19.4666
0.25,-17.0393,-17.2472,-17.3739,-17.4728,-17.5648,-17.6391,-17.6935,-17.732,-17.7889,-17.8182,-17.8546,-17.8929,-17.9183,-17.9562,-17.991
0.5,-16.7826,-16.8047,-16.8195,-16.8477,-16.8722,-16.8972,-16.916,-16.9195,-16.9337,-16.9309,-16.9534,-16.9382,-16.9688,-16.9663,-16.9629
0.75,-16.5119,-16.3661,-16.2833,-16.216,-16.1668,-16.1559,-16.127,-16.0887,-16.0692,-16.0398,-16.0303,-15.9932,-15.9667,-15.959,-15.9434
0.95,-16.1197,-15.6986,-15.45,-15.2957,-15.1573,-15.0575,-14.9613,-14.8696,-14.8242,-14.7759,-14.7094,-14.6436,-14.577,-14.5376,-14.5153
0.98,-15.9597,-15.4214,-15.1348,-14.89,-14.7271,-14.6131,-14.4637,-14.3501,-14.2804,-14.2236,-14.1847,-14.0946,-13.9946,-13.9372,-13.9198


In [79]:
y_data = annualise(scenarios[1, :, 21:25],4)
display("mean")
display(mean(y_data))
display("std")
display(std(y_data))
display("skewness")
display(skewness(y_data))
display("kurtosis")
display(kurtosis(y_data)) 

"mean"

0.07847038577344674

"std"

0.14665736905801155

"skewness"

0.028671210271200467

"kurtosis"

-0.02993304216269843

In [46]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

UndefVarError: UndefVarError: `simulate` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [47]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [48]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


UndefVarError: UndefVarError: `print_percentiles` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [49]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

UndefVarError: UndefVarError: `cov` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [50]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



ArgumentError: ArgumentError: Package Distributions not found in current path.
- Run `import Pkg; Pkg.add("Distributions")` to install the Distributions package.

In [51]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

UndefVarError: UndefVarError: `print_percentiles` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [52]:
returns_summarystats(data_source[:Yeld_10Y],1)

UndefVarError: UndefVarError: `returns_summarystats` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [53]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [54]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [55]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `scenarios` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [56]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [57]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

UndefVarError: UndefVarError: `Normal` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [58]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

UndefVarError: UndefVarError: `Normal` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [59]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

UndefVarError: UndefVarError: `LocationScale` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [60]:
r = rand(Normal(0,3),10_000)
display(skewness(r))
display(kurtosis(r))

UndefVarError: UndefVarError: `Normal` not defined in `Main`
Suggestion: check for spelling errors or missing imports.